In [1]:
import pandas as pd

from src.data_processing.wrangling import cat_conditions, map_comp, map_investimento, map_importo

In [2]:
p01 = pd.read_parquet("data/updated/p01.parquet")
p02 = pd.read_parquet("data/updated/p02.parquet")
p03 = pd.read_parquet("data/updated/p03.parquet")
p04 = pd.read_parquet("data/updated/p04.parquet")
p05 = pd.read_parquet("data/updated/p05.parquet")

componenti = pd.read_parquet("data/updated/components.parquet")
missions = pd.read_parquet("data/updated/missions.parquet")
comuni = pd.read_parquet("data/updated/comuni.parquet")

indicatori_sdg = pd.read_parquet("data/updated/indicators_sdg.parquet")
indicators = pd.read_parquet("data/updated/indicators.parquet")
targets = pd.read_parquet("data/updated/targets.parquet")
indicatori_com_regis = pd.read_parquet("data/updated/universo_regis.parquet")

## Data preprocessing

In [3]:
p01.shape

(4801727, 2)

In [4]:
p01.columns

Index(['CIG', 'CUP'], dtype='object')

In [5]:
p02.columns

Index(['Programma', 'Missione', 'Descrizione Missione', 'Componente',
       'Descrizione Componente', 'ID Misura', 'Codice Univoco Misura',
       'Descrizione Misura', 'ID Submisura', 'Codice CID',
       'Codice Univoco Submisura', 'Descrizione Submisura',
       'Amministrazione Titolare',
       'Codice Identificativo Procedura di Attivazione', 'Titolo Procedura',
       'Tipologia Procedura di Attivazione', 'CUP', 'Codice Locale Progetto',
       'Stato CUP', 'CUP Codice Natura', 'CUP Descrizione Natura',
       'CUP Codice Tipologia', 'CUP Descrizione Tipologia',
       'CUP Codice Settore', 'CUP Descrizione Settore',
       'CUP Codice Sottosettore', 'CUP Descrizione Sottosettore',
       'CUP Codice Categoria', 'CUP Descrizione Categoria', 'Titolo Progetto',
       'Sintesi Progetto', 'Descrizione Tipo Aiuto', 'Finanziamento - Stato',
       'Finanziamento Stato - FOI',
       'Finanziamento Prosecuzione Opere Pubbliche - FPOP',
       'Finanziamento UE (Diverso da PNRR)', 'Fi

In [6]:
p02.shape

(266562, 54)

In [7]:
p02.head()

,Programma,Missione,Descrizione Missione,Componente,Descrizione Componente,ID Misura,Codice Univoco Misura,Descrizione Misura,ID Submisura,Codice CID,...,Altri Fondi,Finanziamento Totale,Finanziamento Totale Pubblico,Finanziamento Totale Pubblico Netto,Soggetto Attuatore,Codice Fiscale Soggetto Attuatore,Flag Progetti in Essere,Data di Estrazione,Progetto Validato,Data di ultima validazione
0,PNRR,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.1,M1C1I1.01,Infrastrutture digitali,M1C1I1.1,M1C1I1.1,...,"0,00","1171320,00","1171320,00","1171320,00",AZIENDA SANITARIA PROVINCIALE DI SIRACUSA,1661590891,Sì,18/04/2024,Sì,20/03/2024
1,PNRR,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.1,M1C1I1.01,Infrastrutture digitali,M1C1I1.1,M1C1I1.1,...,"0,00","926160,00","926160,00","926160,00",AZIENDA SANITARIA PROVINCIALE DI CALTANISSETTA,1825570854,Sì,18/04/2024,Sì,20/03/2024
2,PNRR,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.1,M1C1I1.01,Infrastrutture digitali,M1C1I1.1,M1C1I1.1,...,"0,00","7599960,00","7599960,00","7599960,00",ASL ROMA 2,13665151000,Sì,18/04/2024,Sì,20/03/2024
3,PNRR,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.1,M1C1I1.01,Infrastrutture digitali,M1C1I1.1,M1C1I1.1,...,"0,00","1358595,00","1358595,00","1358595,00",ASL ROMA 5,4733471009,Sì,18/04/2024,Sì,20/03/2024
4,PNRR,M1,"Digitalizzazione, innovazione, competitività e...",M1C1,"Digitalizzazione, innovazione e sicurezza nell...",M1C1I1.1,M1C1I1.01,Infrastrutture digitali,M1C1I1.1,M1C1I1.1,...,"0,00","55318,00","55318,00","55318,00",MINISTERO DELL'INTERNO,97149560589,No,18/04/2024,Sì,20/03/2024


In [8]:
p03 = p03.rename(columns={'cig':'CIG'})
p03['CATEGORIA'] = p03.apply(cat_conditions,axis=1)

In [9]:
print(p03.shape)
print(p03.columns)
p03.head()

(18548, 4)
Index(['CIG', 'cod_mis_premiale', 'misura_premiale', 'CATEGORIA'], dtype='object')


,CIG,cod_mis_premiale,misura_premiale,CATEGORIA
0,A02D3E0D1C,1,NEI TRE ANNI ANTECEDENTI LA DATA DI SCADENZA D...,GENERALE
1,975319926E,1,NEI TRE ANNI ANTECEDENTI LA DATA DI SCADENZA D...,GENERALE
2,975319926E,6,"ABBIA, NELL'ULTIMO TRIENNIO, RISPETTATO I PRIN...",GENERE
3,975319926E,8,"ABBIA, NELL'ULTIMO TRIENNIO, RISPETTATO GLI OB...",DISABILI
4,A028BBA332,1,NEI TRE ANNI ANTECEDENTI LA DATA DI SCADENZA D...,GENERALE


In [10]:
p04 = p04.rename(columns={'cig':'CIG'})
print(p04.shape)
print(p04.columns)
p04.head()

(256878, 6)
Index(['CIG', 'flag_quote', 'quota_femminile', 'quota_giovanile',
       'cod_mot_deroga', 'mot_deroga'],
      dtype='object')


,CIG,flag_quote,quota_femminile,quota_giovanile,cod_mot_deroga,mot_deroga
0,A0178738AE,N,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO
1,9979062678,N,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO
2,9984663490,N,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO
3,A00867451A,N,NaN,NaN,8.0,ALTRO
4,9807735EF2,N,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO


In [11]:
p05['importo_complessivo_gara'].unique()

array(['9800.0', '225000.0', '8280000.0', ..., '1423749.38', '6684.26',
       '27017.57'], dtype=object)

In [12]:
p05 = p05.rename(columns={'cig':'CIG'})
p05['importo_complessivo_gara'] = p05['importo_complessivo_gara'].astype(float)
p05['CLASSE_IMPORTO'] = p05['importo_complessivo_gara'].apply(map_importo)

In [13]:
print(p05.shape)
print(p05.columns)
p05.head()

(210264, 62)
Index(['CIG', 'cig_accordo_quadro', 'numero_gara', 'oggetto_gara',
       'importo_complessivo_gara', 'n_lotti_componenti', 'oggetto_lotto',
       'importo_lotto', 'oggetto_principale_contratto', 'stato', 'settore',
       'luogo_istat', 'provincia', 'data_pubblicazione',
       'data_scadenza_offerta', 'cod_tipo_scelta_contraente',
       'tipo_scelta_contraente', 'cod_modalita_realizzazione',
       'modalita_realizzazione', 'codice_ausa',
       'cf_amministrazione_appaltante',
       'denominazione_amministrazione_appaltante', 'sezione_regionale',
       'id_centro_costo', 'denominazione_centro_costo', 'anno_pubblicazione',
       'mese_pubblicazione', 'cod_cpv', 'descrizione_cpv', 'flag_prevalente',
       'COD_MOTIVO_CANCELLAZIONE', 'MOTIVO_CANCELLAZIONE',
       'DATA_CANCELLAZIONE', 'DATA_ULTIMO_PERFEZIONAMENTO',
       'COD_MODALITA_INDIZIONE_SPECIALI', 'MODALITA_INDIZIONE_SPECIALI',
       'COD_MODALITA_INDIZIONE_SERVIZI', 'MODALITA_INDIZIONE_SERVIZI',
       'D

,CIG,cig_accordo_quadro,numero_gara,oggetto_gara,importo_complessivo_gara,n_lotti_componenti,oggetto_lotto,importo_lotto,oggetto_principale_contratto,stato,...,CUI_PROGRAMMA,FLAG_PREV_RIPETIZIONI,COD_IPOTESI_COLLEGAMENTO,IPOTESI_COLLEGAMENTO,CIG_COLLEGAMENTO,COD_ESITO,ESITO,DATA_COMUNICAZIONE_ESITO,FLAG_PNRR_PNC,CLASSE_IMPORTO
0,9766166F1F,None,9042660,'AVVISO INVESTIMENTO 1.2 'ABILITAZIONE AL CLOU...,9800.0,1,'AVVISO INVESTIMENTO 1.2 'ABILITAZIONE AL CLOU...,9800.0,SERVIZI,ATTIVO,...,00104440144202300001,1,None,None,None,None,None,None,1,BASSA
1,A019FC8DE0,7194294737,9362129,PNRR MISSIONE 6 COMPONENTE 2 INVESTIMENTO 1.1....,225000.0,1,PNRR MISSIONE 6 COMPONENTE 2 INVESTIMENTO 1.1....,225000.0,FORNITURE,ATTIVO,...,0000000000000000000000,0,None,None,None,None,None,None,1,MEDIA
2,819779499C,None,7678766,S.C. 20178924 QUADRI DI BASSA TENSIONE PER CAB...,8280000.0,2,S.C. 20178924 SCOMPARTO DI BASSA TENSIONE TIPO...,3312000.0,FORNITURE,ATTIVO,...,None,1,None,None,None,1.0,AGGIUDICATA,2023-08-29,1,ALTA
3,8226615176,7040391A70,7700533,FORNITURA CUCINE DI REPARTO,50000.0,1,FORNITURA CUCINE DI REPARTO,50000.0,FORNITURE,ATTIVO,...,None,0,None,None,None,1.0,AGGIUDICATA,2021-01-22,1,BASSA
4,82066050B0,None,7686388,GARA GBO PER INAIL - ID 2208,9693890.0,2,LOTTO 2 - GARA GBO PER INAIL ID 2208,1703390.0,SERVIZI,ATTIVO,...,None,0,None,None,None,1.0,AGGIUDICATA,2022-01-18,1,ALTA


## Merge dei Dataset "P0"

In [22]:
# missioni e componenti, left join da p02
p02 = p02.merge(missions,left_on="Missione", right_on=missions.index, how='inner')
p02 = p02.drop_duplicates()
p02 = p02.merge(componenti,left_on="Componente", right_on=componenti.index, how='inner')
p02 = p02.drop_duplicates()

In [16]:
# left join p05 con i dati su comuni e denominazioni 
p05=p05.merge(
    comuni,
    left_on='provincia', 
    right_on='Denominazione sovracomunale',
    how='left'
)
p05=p05.drop_duplicates()

In [25]:
# left join p5 e p3
leftnew_3 = p05.merge(p03, on='CIG', how='left')
leftnew_3 = leftnew_3.drop_duplicates()
print(leftnew_3.shape)

(219714, 67)


In [26]:
# left join il risultato con p4
leftnew_34 = leftnew_3.merge(p04, on='CIG', how='left')
leftnew_34 = leftnew_34.drop_duplicates()
print(leftnew_34.shape)

(232207, 72)


In [27]:
# left join il risultato con p1
leftnew_341 = leftnew_34.merge(p01, on='CIG', how='left')
leftnew_341 = leftnew_341.drop_duplicates()
print(leftnew_341.shape)

(242647, 73)


In [31]:
# left join il risultato con p2
leftnew_3412 = leftnew_341.merge(p02,how='left',left_on='CUP',right_on='CUP')
leftnew_3412 = leftnew_3412.drop_duplicates()
print(leftnew_3412.shape)

(303623, 128)


### Merge Dati ItaliaDomani

In [56]:
colonne_da_selezionare = [
    'CIG', 
    'CUP',  
    'Regione', 
    'provincia',  
    # 'COMUNE', 
    'importo_complessivo_gara', 
    'CLASSE_IMPORTO', 
    'Descrizione Missione', 
    'Missione', 
    'Componente',  
    'cod_mis_premiale', 
    'misura_premiale', 
    'CATEGORIA', 
    'flag_quote', 
    'quota_femminile', 
    'quota_giovanile', 
    'cod_mot_deroga', 
    'mot_deroga', 
    'data_pubblicazione', 
    'anno_pubblicazione', 
    'FLAG_URGENZA', 
    'MOTIVO_URGENZA', 
    'ESITO'
]

In [54]:
for col in leftnew_3412.columns:
    print(col)

CIG
cig_accordo_quadro
numero_gara
oggetto_gara
importo_complessivo_gara
n_lotti_componenti
oggetto_lotto
importo_lotto
oggetto_principale_contratto
stato
settore
luogo_istat
provincia
data_pubblicazione
data_scadenza_offerta
cod_tipo_scelta_contraente
tipo_scelta_contraente
cod_modalita_realizzazione
modalita_realizzazione
codice_ausa
cf_amministrazione_appaltante
denominazione_amministrazione_appaltante
sezione_regionale
id_centro_costo
denominazione_centro_costo
anno_pubblicazione
mese_pubblicazione
cod_cpv
descrizione_cpv
flag_prevalente
COD_MOTIVO_CANCELLAZIONE
MOTIVO_CANCELLAZIONE
DATA_CANCELLAZIONE
DATA_ULTIMO_PERFEZIONAMENTO
COD_MODALITA_INDIZIONE_SPECIALI
MODALITA_INDIZIONE_SPECIALI
COD_MODALITA_INDIZIONE_SERVIZI
MODALITA_INDIZIONE_SERVIZI
DURATA_PREVISTA
COD_STRUMENTO_SVOLGIMENTO
STRUMENTO_SVOLGIMENTO
FLAG_URGENZA
COD_MOTIVO_URGENZA
MOTIVO_URGENZA
FLAG_DELEGA
FUNZIONI_DELEGATE
CF_SA_DELEGANTE
DENOMINAZIONE_SA_DELEGANTE
CF_SA_DELEGATA
DENOMINAZIONE_SA_DELEGATA
IMPORTO_SICUREZZ

In [57]:
master_table = leftnew_3412[colonne_da_selezionare]

In [58]:
master_table

,CIG,CUP,Regione,provincia,importo_complessivo_gara,CLASSE_IMPORTO,Descrizione Missione,Missione,Componente,cod_mis_premiale,...,flag_quote,quota_femminile,quota_giovanile,cod_mot_deroga,mot_deroga,data_pubblicazione,anno_pubblicazione,FLAG_URGENZA,MOTIVO_URGENZA,ESITO
0,9766166F1F,I11C22001120006,Lombardia,SONDRIO,9800.00,BASSA,"Digitalizzazione, innovazione, competitività e...",M1,M1C1,NaN,...,N,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,2020-01-01,2020,0,None,None
1,A019FC8DE0,J89J22002200006,Puglia,BRINDISI,225000.00,MEDIA,Salute,M6,M6C2,NaN,...,None,NaN,NaN,NaN,None,2020-02-10,2020,0,None,None
2,819779499C,F88B22001390006,NaN,None,8280000.00,ALTA,Rivoluzione verde e transizione ecologica,M2,M2C2,NaN,...,None,NaN,NaN,NaN,None,2020-02-28,2020,0,None,AGGIUDICATA
3,8226615176,D19J20000070005,Toscana,FIRENZE,50000.00,BASSA,Salute,M6,M6C2,NaN,...,None,NaN,NaN,NaN,None,2020-02-25,2020,0,None,AGGIUDICATA
4,82066050B0,E51B21004900006,NaN,None,9693890.00,ALTA,"Digitalizzazione, innovazione, competitività e...",M1,M1C1,NaN,...,None,NaN,NaN,NaN,None,2020-02-19,2020,0,None,AGGIUDICATA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303618,A03215F0C9,B45E22000310001,NaN,None,1423749.38,ALTA,Inclusione e coesione,M5,M5C2,7.0,...,S,NaN,NaN,NaN,None,2023-12-06,2023,0,None,None
303619,A03215F0C9,B45E22000310001,NaN,None,1423749.38,ALTA,Inclusione e coesione,M5,M5C2,8.0,...,S,NaN,NaN,NaN,None,2023-12-06,2023,0,None,None
303620,A03A0809C2,E27H21008350006,Campania,SALERNO,6684.26,BASSA,Inclusione e coesione,M5,M5C3,NaN,...,N,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,2023-12-07,2023,0,None,None
303621,A03A0809C2,E27H21008350006,Campania,SALERNO,6684.26,BASSA,Inclusione e coesione,M5,M5C3,NaN,...,N,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,2023-12-07,2023,0,None,None


In [59]:
master_table.to_csv("data/updated/master_table.csv", sep=";")